# Linear Regression in TensorFlow

This exercise takes you through the fundamental linear regression model from a number of different angles. First we consider an analytic analysis and then we continue to consider how to solve the same problem using numerical methods. This lab also serves and an introduction to the tensorflow toolkit which will be useful for a variety of machine learning tasks in the future and is used by Google to solve massive machine learning problems on their clusters.

The linear regression model forms the basis for a whole host of models - if you are comfortable with the fundamental approaches we take here, there will be a whole range of extensions, advances and applications available to you in the future.

For this lab exercise there are 6 places where you are expected to enter your own code. Every place you have to add code is indicated by

`# Add your code here ..`

`# ***********************************************************`

with instructions above the code block.

In [0]:
%matplotlib inline
import numpy as np

# A new one for this lab!
import tensorflow as tf

from sklearn import datasets as ds

import matplotlib.pyplot as plt

In [0]:
boston = ds.load_boston()

x_raw = boston.data[:,np.argwhere(boston.feature_names == 'RM')[0,0]]
y_raw = boston.target

## Boston House Prices Dataset
A descroption of the dataset used is provided here.

Data Set Characteristics:  
    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in $1000's

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
http://archive.ics.uci.edu/ml/datasets/Housing


This dataset was taken from the StatLib library which is maintained at Carnegie Mellon University.

The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
prices and the demand for clean air', J. Environ. Economics & Management,
vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
...', Wiley, 1980.   N.B. Various transformations are used in the table on
pages 244-261 of the latter.

The Boston house-price data has been used in many machine learning papers that address regression
problems.   
     
### References

   - Belsley, Kuh & Welsch, 'Regression diagnostics: Identifying Influential Data and Sources of Collinearity', Wiley, 1980. 244-261.
   - Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning. In Proceedings on the Tenth International Conference of Machine Learning, 236-243, University of Massachusetts, Amherst. Morgan Kaufmann.
   - many more! (see http://archive.ics.uci.edu/ml/datasets/Housing)

## Preparing the data

We are going to look at the relationship between the "average number of rooms per dwelling" and median house price in the Boston dataset. First let us partition the data into a training and test split. We are going for 60% training and 40% testing.

In [0]:
total_count = x_raw.shape[0]

split = int(total_count * 0.6)

# Shuffle the data to avoid any ordering bias..
np.random.seed(0)
shuffle = np.random.permutation(total_count)

x = x_raw[shuffle]
y = y_raw[shuffle]

x_train_unnormalised = x[:split]
y_train_unnormalised = y[:split]

x_test_unnormalised = x[split:]
y_test_unnormalised = y[split:]

print('Training set size:', x_train_unnormalised.shape[0])
print('Test set size:', x_test_unnormalised.shape[0])

## Data Visualisation

To allow for easy visualisation as you progress through the task we are using a single dimensional data set. Both the input $x$ and output $y$ are scalars so we can plot them on a standard scatter plot.

In [0]:
# You can use this function to plot the data and then add your own plots on top..
def plot_data(x, y):
    plt.figure(figsize=[10,8])
    plt.plot(x, y, 'o')
    plt.grid(True)
    plt.xlabel('Average number rooms per dwelling')
    plt.ylabel('Mean value of home')

plot_data(x_train_unnormalised, y_train_unnormalised)
plt.title('Plot of the Training Data')

### Task 1:

Write a function that normalises a vector of values. It should output a corresponding vector where the values have a mean of zero and a standard deviation of 1. Note that you should only perform an affine transformation of the data (i.e. a linear scaling and a fixed offset). This means that you must find $a$ and $b$ for $v_i = a u_i + b$ where $u$ is the input data and $v$ is the normalised output data.

Your function should return the normalising constants as well as the normalised data.

Write a second function that removes the normalisation and returns the data to its original values.

Check that passing both `x_train` and `y_train` through both functions returns the vectors to their original values.

*Hints:*
- You might want to look at `np.all()` for the Boolean check that they return to their values.
- When checking that floating point values are equal up to nummerical precision (e.g. rounding errors in the computations) you can use the `np.isclose()` function.
- You can use the `assert()` command to guarantee that a statement is `True` before the program continues.

*Points to consider:* 
- Why might it be sensible to normalise the data in the fashion described?
- Considering that we are about to perform Linear Regression, why might we not want to perform a more involved normalisation process?

In [0]:
def normalise_data(x_unnormalised):
    # Add your code here..
    # ************************************************************
    # ...
    return x_normalised, a, b

def unnormalise_data(x_normalised, a, b):
    # Add your code here..
    # ************************************************************
    # ...
    return x_unnormalised

x_train, x_norm_a, x_norm_b = normalise_data(x_train_unnormalised)
y_train, y_norm_a, y_norm_b = normalise_data(y_train_unnormalised)

x_test, _, _ = normalise_data(x_test_unnormalised)
y_test, _, _ = normalise_data(y_test_unnormalised)

# Add your code here to check that the unnormaliseding the 
# training data returns to their original values..
# ************************************************************
# ...

# Plot the data to make sure they are normalised..
plot_data(x_train, y_train)
plt.title('Plot of the (Normalised) Training Data')

## The Linear Regression Model

In linear regression we are trying to fit a linear model to the data of the form

\begin{align}
y &= w x + c
\end{align}

where $w$ and $c$ are parameters to be learned that take the input data $x$ to the output data $y$. Once this model has been learned, we can use the parameters to predict the values of the output that would correspond to new values of the input.

In order to determine the parameters, we need an objective function that we seek to optimise: this function returns a scalar value for all possible parameter values and we seek to change the parameters until the best scalar value is obtained.

For linear regression, we usually take the objective as one which minimises the squared error; this is known as a linear least squares problem.

*Aside: Think about what this means in terms of a model for the data when you have $y = f(x) + \eta$ with $f(x)$ as a linear function $f(x) = w x + c$ and $\eta$ as iid Gaussian noise.*

Therefore our objective is given by the sum of squared differences between the true value of $y_i$ and the value estimated by our model $w x_i + c$.

\begin{align}
E(w,c) &= \sum_{i=0}^{N-1} \big(y_i - f(x_i) \big)^2 \\
    &= \sum_{i=0}^{N-1} \big(y_i - (w x_i + c) \big)^2 \\
    &= \sum_{i=0}^{N-1} \big(y_i - w x_i - c \big)^2
\end{align}



## Task 2

Write a function that calculates the least squared error on the training data for a particular value of the parameters $w$ and $c$.

In [0]:
def least_squares_error(x, y, w, c):
    # Add code to calcuate the squared_error = E(w,c)..
    # ************************************************************
    # ...
    return squared_error

print('Squared error for w = 1.5, c = 0.5 is ', 
      least_squares_error(x_train, y_train, w=1.5, c=0.5))

## Analytic Solution

In the case of linear regression, we can find an analytic solution to this problem by finding stationary point of the objective function. We do this by evaluating the partial derivatives of the objective wrt each parameter in turn and setting them to zero. If we can then find a solution to these simultaneous equations, we have found an optimal setting for the parameters.

For $w$ we have:

\begin{align}
\frac{\partial E}{\partial w} 
    &= \frac{\partial}{\partial w}\sum_{i=0}^{N-1} \big(y_i - w x_i - c \big)^2 \\
    &= \sum_{i=0}^{N-1} \frac{\partial}{\partial w} \big(y_i - w x_i - c \big)^2 \\
    &= \sum_{i=0}^{N-1} 2 \big(y_i - w x_i - c \big) \frac{\partial}{\partial w} \big(y_i - w x_i - c \big) \\
    &= \sum_{i=0}^{N-1} 2 \big(y_i - w x_i - c \big) \big(- x_i \big) \\
    &= \sum_{i=0}^{N-1} 2 x_i (w x_i + c - y_i) \\
\end{align}

For $c$ we have:

\begin{align}
\frac{\partial E}{\partial c} 
    &= \frac{\partial}{\partial c}\sum_{i=0}^{N-1} \big(y_i - w x_i - c \big)^2 \\
    &= \sum_{i=0}^{N-1} \frac{\partial}{\partial c} \big(y_i - w x_i - c \big)^2 \\
    &= \sum_{i=0}^{N-1} 2 \big(y_i - w x_i - c \big) \frac{\partial}{\partial c} \big(y_i - w x_i - c \big) \\
    &= \sum_{i=0}^{N-1} 2 \big(y_i - w x_i - c \big) \big(- 1 \big) \\
    &= \sum_{i=0}^{N-1} 2 (w x_i + c - y_i) \\
\end{align}

Now setting $\frac{\partial E}{\partial w} = 0$:

\begin{align}
\Rightarrow \sum_{i=0}^{N-1} x_i (w x_i + c - y_i) &= 0 \\
w \sum_{i=0}^{N-1} (x_i)^2 + c \sum_{i=0}^{N-1} (x_i) - \sum_{i=0}^{N-1} (x_i y_i) &= 0 \\
\Rightarrow A w + B c - C &= 0
\end{align}

with $A = \sum_{i=0}^{N-1} (x_i)^2$, $B = \sum_{i=0}^{N-1} (x_i)$ and $C = \sum_{i=0}^{N-1} (x_i y_i)$.

Now setting $\frac{\partial E}{\partial c} = 0$:

\begin{align}
\Rightarrow \sum_{i=0}^{N-1} (w x_i + c - y_i) &= 0 \\
w \sum_{i=0}^{N-1} (x_i) + N c - \sum_{i=0}^{N-1} (y_i) &= 0 \\
\Rightarrow B w + N c - D &= 0
\end{align}

with $D = \sum_{i=0}^{N-1} (y_i)$.

Combining these two:

\begin{align}
\Rightarrow c = \frac{1}{N} (D - B w) \\
\Rightarrow A w + \frac{B}{N} (D - B w) - C = 0 \\
\Rightarrow w \left(A - \frac{B^2}{N}\right) = \left(C - \frac{BD}{N}\right) \\
\Rightarrow w = \frac{CN - BD}{AN - B^2} \\
\Rightarrow c = \frac{1}{N} \left(D - B \frac{\left(CN - BD\right)}{\left(AN - B^2\right)}\right) \\
\Rightarrow c = \frac{1}{N} \left(\frac{D \left(AN - B^2\right) - B C N + B^2 D}{\left(AN - B^2\right)}\right) \\
\Rightarrow c = \frac{AD - BC}{AN - B^2}
\end{align}

## Task 3

Write a function using numpy that takes as input `x_train` and `y_train` and outputs the optimal $w$ and $c$ parameters for least squares linear regression.

Confirm that your results are reasonable by plotting the resulting linear function on the training graph as well as the predicted values for the test set on a separate graph.

In [0]:
def least_squares_analytic_solution(x, y):
    # Add code to calcuate the optimal w and c ..
    # ************************************************************
    # ...
    return w, c

w_opt, c_opt = least_squares_analytic_solution(x_train, y_train)

print('Analytic solution:')
print('Analytic w = ', w_opt)
print('Analytic c = ', c_opt)

plot_data(x_train, y_train)
plt.title('Analytic Linear Regression (Training Data)')
# Add code to plot a line showing your solution 
# for w and c..
# ************************************************************
# ...

plot_data(x_test, y_test)
plt.title('Analytic Linear Regression (Testing Data)')
# Add code to plot your predictions for the x_test data
# for comparison against y_test..
# ************************************************************
# ...

print('Mean least squares error on TRAINING data = ',
     least_squares_error(x_train, y_train, w_opt, c_opt) / x_train.shape[0])

print('Mean least squares error on TEST data = ',
     least_squares_error(x_test, y_test, w_opt, c_opt) / x_test.shape[0])

## Tensorflow Optimisation

**For help with this please refer to the separate tensorflow example that was worked through in class and is available on moodle as a separate jupyter notebook. Please load the example notebook and run through it yourself again before continuing with this task.**

In the previous task we looked at an analytic solution to the least square problem. We now pretend that we could not solve the problem analytically. Although this is not true for this case, the additional of extensions to the linear regression model (for example to improve robustness, adding feature selection or handling non-linear data) can mean that it is no longer possible to find an analytic solution and numerical optimisation must be used.

## Task 4

Add the tensorflow expressions to the following code to calculate the least squares error using tensorflow and check that it calculates the same value as the numpy version.


In [0]:
tf.reset_default_graph()

# Initial values for optimisation..
w_initial_guess = 1.5
c_initial_guess = 0.5

# Constants to hold the training data..
t_x_train = tf.constant(x_train, name='x_train')
t_y_train = tf.constant(y_train, name='y_train')

# Variables to hold w and c
t_w = tf.Variable(w_initial_guess, 
                  dtype=tf.float64, 
                  name='w')
t_c = tf.Variable(c_initial_guess, 
                  dtype=tf.float64, 
                  name='c')

# Add your code here to calculate t_least_squares_error..
# ************************************************************
# ...

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    tensorflow_result = session.run(t_least_squares_error)

# Check with the result from your previous function..
numpy_result = least_squares_error(x_train, y_train, 
                                   w=w_initial_guess, 
                                   c=c_initial_guess)

print('Tensorflow least squares error = ', tensorflow_result)
print('numpy least squares error = ', numpy_result)

# This should pass if they are the same to nummerical precision!
assert(np.isclose(tensorflow_result, numpy_result))

## Task 5

Check that the gradients from tensorflow are correct by writing numpy code to calculate the value of the derivatives from the analytic expressions (derived previously):

\begin{align}
\frac{\partial E}{\partial w} 
    &= \sum_{i=0}^{N-1} 2 x_i (w x_i + c - y_i) \\
\end{align}

\begin{align}
\frac{\partial E}{\partial c} 
    &= \sum_{i=0}^{N-1} 2 (w x_i + c - y_i) \\
\end{align}

In [0]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    t_gradient_wrt_w = tf.gradients(t_least_squares_error, t_w)
    t_gradient_wrt_c = tf.gradients(t_least_squares_error, t_c)
    
    tf_grad_w = session.run(t_gradient_wrt_w)
    tf_grad_c = session.run(t_gradient_wrt_c)
    
    print('Tensorflow gradient wrt w = ', tf_grad_w)
    print('Tensorflow gradient wrt c = ', tf_grad_c)

def calc_gradients_for_least_squares(x, y, w, c):
    # Add your code to evalute the partial derivatives here
    # ************************************************************
    # ...
    return grad_w, grad_c

numpy_grad_w, numpy_grad_c = calc_gradients_for_least_squares(x_train, 
                                                              y_train, 
                                                              w_initial_guess, 
                                                              c_initial_guess)

print('Analytic gradient wrt w = ', numpy_grad_w)
print('Analytic gradient wrt c = ', numpy_grad_c)

# This should pass if they are the same to nummerical precision!
assert(np.isclose(tf_grad_w, numpy_grad_w))
assert(np.isclose(tf_grad_c, numpy_grad_c))

## Gradient Descent in Tensorflow

If all has gone well, the gradients should be the same and you can use tensorflow to find the solution to the optimisation.

**Run the following block of code to see the tensorflow optimisation running with your least square error function from above!**

*Aside: Investigate what happens as you change the `learning_rate` parameter as well as the `num_iterations`. Can we guarantee that tensorflow will always return the same result as the analytic solution? What might be happening if not?*

In [0]:
# Create a gradient descent optimiser that uses a
# certain step size (learning_rate)..
optimiser = tf.train.GradientDescentOptimizer(learning_rate=0.001)

# We want to minimise the least squares error
t_objective = t_least_squares_error

# We want to optimise wrt w and c
vars_to_optimise = [t_w, t_c]

minimize_operation = optimiser.minimize(t_objective,
                                        var_list=vars_to_optimise)

# Number of iterations to perform
num_iterations = 15

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    # Run a number of iterations of gradient descent..
    for iteration in range(num_iterations):
        # At each iteration evaluate the minimize_operation
        # to perform the gradient descent step and also
        # keep track of the current value..
        step, cost = session.run([minimize_operation, 
                                  t_least_squares_error])
        
        # Print out the value of log P every 10 iterations..
        #if ((iteration + 1) % 10 == 0):
        print('iter %4d, E(w,c) = %0.3f' % 
                  (iteration + 1, cost))
    
    # Get the final results of the optimisation..
    w_tf_opt = session.run(t_w)
    c_tf_opt = session.run(t_c)
    
    print('\nAfter tensorflow optimisation:')
    print('Tensorflow w = ', w_tf_opt)
    print('Tensorflow c = ', c_tf_opt)

print('\nAnalytic solution:')
print('Analytic w = ', w_opt)
print('Analytic c = ', c_opt)

## Task 6

You are now going to implement your own version of gradient descent (the process that tensorflow is providing to perform optimisation). The good news is that you already have the numpy functions to calculate both the least squares error (`least_squares_error`) and the gradients wrt the parameters (`calc_gradients_for_least_squares`).

Starting from the same initial values as tensorflow (`w_initial_guess`, `c_initial_guess`) write an iterative algorithm for gradient descent. At each iteration it should perform the following steps:

- Evaluate and save the squared error for the current parameters
- Calculate the gradients wrt w and c for the current parameters
- Update the parameters for w and c my moving in the direction of the negative current proportional to the current step size

**The code below should run with these three additions. Once they are working, move on to the following.**

Under this algorithm, the squared error should decrease at each iteration. If it is not decreasing then either there is a bug in the code (!) or the step size is too large. Add a check that makes sure the squared error always decreases and if it fails to decrease, decrease the step size and run the iteration again.

**Try running this code starting with `current_step_size = 0.1`.**

You can improve your answer by checking to see if you should stop iterating. If the change in the squared error between successive iterations is very small then one of the following is true. Either the step size is too small (not changing the parameters sufficiently) or the values have converged to their optimal values. If you cannot find a step size that creates a deacrease in the squared error then you have probably converged and can stop performing iterations.

*Hint: you can exit a `for` loop early with the `break` command.*

**Try running this code starting with `num_iterations = 200`.**

In [0]:
# Keep track of parameter values over iterations..
w_current = w_initial_guess
c_current = c_initial_guess

# Keep track of the error..
E_current = least_squares_error(x_train, y_train, w_current, c_current)

# Keep track of the step size..
current_step_size = 0.001

num_iterations = 20

for iteration in range(num_iterations):
    
    # Add code to evaluate the gradients..
    # ************************************************************
    # ...
    
    # Add code to take a step in the direction of the negative
    # gradient proportional to the step size..
    # ************************************************************
    # ...
    w_new =
    c_new = 
    
    # Add code to evaluate and remember the squared error..
    # ************************************************************
    # ...
    E_new = 
    
    # Add code to check that error is decreasing and reduce step
    # size if not..
    # ************************************************************
    # ...

    # Add code to check for convergence and terminate
    # the loop if converged..
    # ************************************************************
    # ...
    
    print('iteration %4d, E = %f, w = %f, c = %f' % 
          (iteration, E_current, w_new, c_new))
    
    # Take the step
    w_current = w_new
    c_current = c_new
    E_current = E_new

print('\nAfter gradient descent optimisation:')
print('Optimised w = ', w_current)
print('Optimised c = ', c_current)

print('\nAnalytic solution:')
print('Analytic w = ', w_opt)
print('Analytic c = ', c_opt)